In [6]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [61]:
#-------------------GEOMETRY-----------------#
#per ogni coordinata definisco il passo in avanti o indietro
#con le opportune condizioni al bordo

def geometry(nlatt, nvol):
    
    npp = np.zeros(nlatt)
    nmm = np.zeros(nlatt)
    
    for i in range(0,nlatt):
        npp[i] = i + 1
        nmm[i] = i - 1
    
    npp[-1] = 1
    nmm[0] = nlatt
    
    return npp,nmm

###da 0 a nlatt o da 1 a nlatt??###

In [62]:
#------------------LATTICE-----------------------#
# assegno la configurazione di partenza della catena di Markov

def initialize_lattice(nlatt, nvol, iflag ,seed, field_start):
    
    field = np.ones((nlatt,nlatt))
    #PARTENZA A FREDDO (tutti gli spin a 1 come se fosse T = 0)
    if iflag == 0 :
        field = np.ones((nlatt,nlatt))
    #... A CALDO ... (spin random, come se fosse T = infinito) 
    elif iflag ==1 :
        for i in range(0, nlatt):
            for j in range(0, nlatt):
                x = np.random.random()
                field[i][j] = 1
                if x < 0.5 :
                    field[i][j] = -1.0
    #... O DA DOVE ERO RIMASTO L'ULTIMA VOLTA
    else:
        field = field_start
        
    return field

In [63]:
#----------------MAGNETIZATION-----------------#
#calcolo magnetizzazione media del reticolo

def magnetization(nlatt, nvol, field):
    
    xmagn = np.sum(field)
    xmagn = xmagn/nvol
    
    return xmagn

In [64]:
#-----------------ENERGY--------------------#
#energia media (= 0 per configurazione ordinata e campo esterno 0)

def energy(nlatt, nvol, field, npp, nmm, extfield):
    
    xene = 0.0
    for i in range(0, nlatt):
        for j in range(0, nlatt):
            ip = npp[i]
            im = nmm[i]
            jp = npp[j]
            jm = nmm[j]
            
            force = field[i][jp] + field[i][jm] + field[ip][j] + field[im][j]
            xene = xene - 0.5 * force * field[i][j]
            xene = xene - extfield*field[i][j]
            
    xene = xene/nvol
    
    return xene

In [65]:
#---------------UPDATE METROPOLIS----------------#
#faccio aggiornamenti locali delle variabili di spin con metropolis
#la variabile di spin di prova e` sempre quella opposta a quella attuale

def update_metropolis(nlatt, nvol, field, npp, nmm, beta, extfield, seed):
    
    for ivol in range(1, nlatt*nlatt):
        i = int(np.random.random()*nlatt + 1.0)
        j = int(np.random.random()*nlatt + 1.0)
        
        ip = npp[i]         
        im = nmm[i]         
        jp = npp[j]         
        jm = nmm[j]
        
        force = field[i][jp] + field[i][jm] + field[ip][j] + field[im][j]
        force = beta*(force + extfield)
        
        phi = field[i][j]
        
        p_rat = np.exp(-2*phi*force)
        
        x = np.random.random()
        
        if x < p_rat :
            field[i][j] = -phi
            
    return field
    

In [68]:
#-------------main------------------#
nlatt = 10
nvol = nlatt**2
seed = 100
beta = 0.3
extfield = 0
iflag = 0
i_decorrel = 100
measures = 100000

field_start = np.empty((nlatt,nlatt))

npp, nmm = geometry(nlatt, nvol)
field = initialize_lattice(nlatt, nvol, iflag ,seed, field_start)

#ma queste ultime operazioni si fanno per ogni valore di i (quindi xmagn[i], xene[i] ecc) o ci interessa solo l'ultimo valore?

for i in range(measures):
    for idec in range(i_decorrel):
        field = update_metropolis(nlatt, nvol, field, npp, nmm, beta, extfield, seed)
        idec = idec + 1
    xmagn = magnetization(nlatt, nvol, field)
    xene = energy(nlatt, nvol, field, npp, nmm, extfield)
    i = i+1
    
xmagn, xene

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices